# 🎯 리셀러 자동 판단 시스템

## 📌 사용법 (매우 간단!)

### 1단계: 파일 업로드
👉 아래 **1️⃣ 파일 업로드** 셀 실행 → 파일 선택

### 2단계: 분석 실행  
👉 **2️⃣ 전체 분석 실행** 셀 클릭

### 3단계: 결과 다운로드
👉 자동으로 엑셀 파일 다운로드됨

---
**주의**: Google Colab에서만 작동합니다!

In [ ]:
# 2️⃣ 전체 분석 실행 (▶️ 클릭!)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.formatting.rule import DataBarRule, ColorScaleRule
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.drawing.image import Image
import io
from PIL import Image as PILImage

# 한글 폰트 설정 (Colab 환경용)
try:
    # Colab에서 나눔고딕 설치
    import subprocess
    subprocess.run(['apt-get', 'install', '-y', 'fonts-nanum'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 폰트 캐시 재생성
    import matplotlib.font_manager as fm
    fm._rebuild()

    # 한글 폰트 설정
    plt.rcParams['font.family'] = 'NanumGothic'
    plt.rcParams['axes.unicode_minus'] = False
    print("✅ 한글 폰트 설정 완료 (NanumGothic)")
except:
    # 폰트 설치 실패시 대체 방안
    try:
        plt.rcParams['font.family'] = 'Malgun Gothic'
    except:
        plt.rcParams['font.family'] = 'AppleGothic'
    plt.rcParams['axes.unicode_minus'] = False
    print("⚠️ 기본 폰트 사용 (한글이 깨질 수 있음)")

print("=" * 60)
print("🚀 프로페셔널 리셀러 분석을 시작합니다...")
print("=" * 60)

# 1. 데이터 읽기
try:
    if not os.path.exists('erp_order_form.xlsx'):
        print("❌ 파일이 없습니다! 먼저 위의 파일 업로드를 하세요.")
    else:
        # 오전/오후 데이터 통합
        df_morning = pd.read_excel('erp_order_form.xlsx', sheet_name='오전')
        df_afternoon = pd.read_excel('erp_order_form.xlsx', sheet_name='오후')
        df = pd.concat([df_morning, df_afternoon], ignore_index=True)
        
        print(f"✅ 데이터 로드 완료: {len(df):,}건")
        print(f"📊 오전: {len(df_morning)}건 + 오후: {len(df_afternoon)}건")
        print(f"👥 총 고객 수: {df['주문자명'].nunique():,}명\n")
        
        # 2. 함수 정의
        def detect_business_address(address):
            """사업장 주소 감지"""
            if pd.isna(address):
                return 0
            address_str = str(address)
            business_keywords = ['사무실', '오피스', '빌딩', '타워', '센터', '상가', '층',
                                '호실', '공장', '창고', '물류', '유통', '산업']
            for keyword in business_keywords:
                if keyword in address_str:
                    return 1
            return 0

        def analyze_reseller_patterns(df):
            """리셀러 패턴 종합 분석"""
            df['주문일자'] = pd.to_datetime(df['주문일자'], errors='coerce')
            
            analysis = df.groupby('주문자명').agg({
                '주문수량': ['sum', 'mean', 'std'],
                '총매출액': 'sum',
                '주문번호(쇼핑몰)': 'count',
                '상품명': ['nunique', lambda x: ', '.join(x.value_counts().head(3).index)],
                '주소': lambda x: x.mode()[0] if len(x) > 0 else '',
                '주문일자': ['min', 'max', lambda x: (x.max() - x.min()).days if len(x) > 1 else 0]
            }).round(2)
            
            analysis.columns = ['총구매수량', '평균구매수량', '수량편차', '총매출액', '주문횟수',
                                '구매제품종류', '주력상품TOP3', '대표주소',
                                '첫구매일', '최근구매일', '활동기간(일)']
            
            analysis['사업장주소'] = analysis['대표주소'].apply(detect_business_address)
            analysis['평균구매주기'] = (analysis['활동기간(일)'] / analysis['주문횟수']).replace([float('inf'), -float('inf')], 0).round(1)
            
            return analysis

        def calculate_professional_reseller_score(stats):
            """전문 리셀러 점수 계산 (5대 지표)"""
            scores = pd.DataFrame(index=stats.index)
            
            # 1. 대량 구매 지표 (30점)
            scores['대량구매'] = stats['평균구매수량'].apply(
                lambda x: 30 if x >= 5 else 20 if x >= 3 else 10 if x >= 2 else 0
            )
            
            # 2. 정기성 지표 (25점)
            scores['정기구매'] = stats.apply(
                lambda x: 25 if (x['주문횟수'] >= 5 and x['활동기간(일)'] >= 30) else
                          15 if (x['주문횟수'] >= 3 and x['활동기간(일)'] >= 14) else
                          5 if x['주문횟수'] >= 2 else 0,
                axis=1
            )
            
            # 3. 사업장 주소 (20점)
            scores['사업장'] = stats['사업장주소'] * 20
            
            # 4. 제품 다양성 (15점)
            scores['다양성'] = stats['구매제품종류'].apply(
                lambda x: 15 if x >= 5 else 10 if x >= 3 else 5 if x >= 2 else 0
            )
            
            # 5. 구매 규모 (10점)
            scores['구매규모'] = stats['총매출액'].apply(
                lambda x: 10 if x >= 200000 else 7 if x >= 100000 else 3 if x >= 50000 else 0
            )
            
            scores['리셀러점수'] = scores.sum(axis=1)
            scores['판정'] = scores['리셀러점수'].apply(
                lambda x: '🔴 확실한 리셀러' if x >= 70 else
                          '🟠 리셀러 가능성 높음' if x >= 50 else
                          '🟡 잠재 리셀러' if x >= 30 else
                          '🟢 일반 고객'
            )
            scores['등급'] = scores['리셀러점수'].apply(
                lambda x: 'A' if x >= 70 else 'B' if x >= 50 else 'C' if x >= 30 else 'D'
            )
            
            final = pd.concat([stats, scores], axis=1)
            return final

        def create_chart_images(final_results):
            """대시보드 차트 이미지 생성"""
            
            # 4개 차트를 개별적으로 생성하여 이미지로 저장
            chart_files = []
            
            # 1. 등급별 분포 파이 차트
            fig1, ax1 = plt.subplots(figsize=(6, 5))
            grade_counts = final_results['등급'].value_counts()
            colors = ['#FF6B6B', '#FFA500', '#FFD700', '#90EE90']
            wedges, texts, autotexts = ax1.pie(
                grade_counts.values, 
                labels=[f'{k}등급\n({v}명)' for k, v in grade_counts.items()], 
                colors=colors, 
                autopct='%1.1f%%', 
                startangle=90,
                textprops={'fontsize': 10}
            )
            ax1.set_title('리셀러 등급 분포', fontsize=14, fontweight='bold', pad=20)
            plt.tight_layout()
            chart1_path = 'chart1_grade_distribution.png'
            plt.savefig(chart1_path, dpi=150, bbox_inches='tight')
            chart_files.append(chart1_path)
            plt.close()
            
            # 2. TOP 10 리셀러 점수 막대 그래프
            fig2, ax2 = plt.subplots(figsize=(8, 6))
            top10_chart = final_results.sort_values(
                ['리셀러점수', '총매출액', '주문횟수'],
                ascending=[False, False, False]
            ).head(10)
            
            bars = ax2.barh(range(len(top10_chart)), top10_chart['리셀러점수'], color='steelblue')
            ax2.set_yticks(range(len(top10_chart)))
            ax2.set_yticklabels(top10_chart.index, fontsize=9)
            ax2.set_xlabel('리셀러 점수', fontsize=12)
            ax2.set_title('TOP 10 리셀러 점수', fontsize=14, fontweight='bold', pad=20)
            ax2.invert_yaxis()
            
            # 막대에 점수 표시
            for i, (bar, score) in enumerate(zip(bars, top10_chart['리셀러점수'])):
                ax2.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, 
                        f'{score:.0f}점', va='center', fontsize=9)
            
            plt.tight_layout()
            chart2_path = 'chart2_top10_scores.png'
            plt.savefig(chart2_path, dpi=150, bbox_inches='tight')
            chart_files.append(chart2_path)
            plt.close()
            
            # 3. 매출 vs 주문횟수 산점도
            fig3, ax3 = plt.subplots(figsize=(8, 6))
            scatter = ax3.scatter(
                final_results['총매출액']/10000, 
                final_results['주문횟수'], 
                c=final_results['리셀러점수'], 
                cmap='RdYlGn', 
                alpha=0.7, 
                s=60,
                edgecolors='black',
                linewidth=0.5
            )
            ax3.set_xlabel('총매출액 (만원)', fontsize=12)
            ax3.set_ylabel('주문횟수', fontsize=12)
            ax3.set_title('매출 vs 주문횟수 (색상: 리셀러 점수)', fontsize=14, fontweight='bold', pad=20)
            ax3.grid(True, alpha=0.3)
            
            # 컬러바 추가
            cbar = plt.colorbar(scatter, ax=ax3)
            cbar.set_label('리셀러 점수', fontsize=10)
            
            plt.tight_layout()
            chart3_path = 'chart3_sales_vs_orders.png'
            plt.savefig(chart3_path, dpi=150, bbox_inches='tight')
            chart_files.append(chart3_path)
            plt.close()
            
            # 4. 등급별 점수 구성 비교 막대 그래프
            fig4, ax4 = plt.subplots(figsize=(8, 6))
            score_components = ['대량구매', '정기구매', '사업장', '다양성', '구매규모']
            
            # A/B등급과 C등급의 평균 점수 계산
            ab_grade = final_results[final_results['등급'].isin(['A', 'B'])]
            c_grade = final_results[final_results['등급'] == 'C']
            
            if len(ab_grade) > 0:
                ab_grade_avg = ab_grade[score_components].mean()
            else:
                ab_grade_avg = pd.Series([0]*5, index=score_components)
                
            if len(c_grade) > 0:
                c_grade_avg = c_grade[score_components].mean()
            else:
                c_grade_avg = pd.Series([0]*5, index=score_components)
            
            x = range(len(score_components))
            width = 0.35
            
            bars1 = ax4.bar([i - width/2 for i in x], ab_grade_avg, width, 
                            label='A/B등급', color='#FF6B6B', alpha=0.8)
            bars2 = ax4.bar([i + width/2 for i in x], c_grade_avg, width, 
                            label='C등급', color='#FFD700', alpha=0.8)
            
            ax4.set_xticks(x)
            ax4.set_xticklabels(score_components, rotation=45, ha='right')
            ax4.set_ylabel('평균 점수', fontsize=12)
            ax4.set_title('등급별 점수 구성 비교', fontsize=14, fontweight='bold', pad=20)
            ax4.legend()
            ax4.grid(True, alpha=0.3, axis='y')
            
            # 막대에 값 표시
            for bars in [bars1, bars2]:
                for bar in bars:
                    height = bar.get_height()
                    if height > 0:
                        ax4.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                                f'{height:.1f}', ha='center', va='bottom', fontsize=9)
            
            plt.tight_layout()
            chart4_path = 'chart4_score_comparison.png'
            plt.savefig(chart4_path, dpi=150, bbox_inches='tight')
            chart_files.append(chart4_path)
            plt.close()
            
            return chart_files

        def create_premium_excel_report(final_results):
            """프리미엄 엑셀 리포트 생성 (시각화 포함)"""
            
            # 차트 이미지 생성
            print("📊 차트 이미지 생성 중...")
            chart_files = create_chart_images(final_results)
            
            wb = Workbook()
            
            # ===== 시트1: 대시보드 =====
            ws_dashboard = wb.active
            ws_dashboard.title = "📊 대시보드"
            
            header_fill = PatternFill(start_color="1F4E78", end_color="1F4E78", fill_type="solid")
            header_font = Font(color="FFFFFF", bold=True, size=14)
            
            ws_dashboard['A1'] = "🎯 리셀러 판단 분석 대시보드"
            ws_dashboard['A1'].font = Font(bold=True, size=20)
            ws_dashboard.merge_cells('A1:H1')
            
            # 통계 요약
            total = len(final_results)
            a_grade = len(final_results[final_results['등급'] == 'A'])
            b_grade = len(final_results[final_results['등급'] == 'B'])
            c_grade = len(final_results[final_results['등급'] == 'C'])
            
            ws_dashboard['A3'] = "📈 전체 통계"
            ws_dashboard['A3'].font = Font(bold=True, size=14)
            
            stats_data = [
                ["구분", "고객수", "비율", "상태"],
                [f"🔴 A등급 (확실한 리셀러)", a_grade, f"{a_grade/total*100:.1f}%", "즉시 연락"],
                [f"🟠 B등급 (가능성 높음)", b_grade, f"{b_grade/total*100:.1f}%", "확인 필요"],
                [f"🟡 C등급 (잠재 리셀러)", c_grade, f"{c_grade/total*100:.1f}%", "관찰 대상"],
                [f"🟢 D등급 (일반 고객)", total-a_grade-b_grade-c_grade, 
                 f"{(total-a_grade-b_grade-c_grade)/total*100:.1f}%", "일반 관리"]
            ]
            
            for r_idx, row in enumerate(stats_data, start=5):
                for c_idx, value in enumerate(row, start=1):
                    cell = ws_dashboard.cell(row=r_idx, column=c_idx, value=value)
                    if r_idx == 5:
                        cell.fill = header_fill
                        cell.font = header_font
                    cell.alignment = Alignment(horizontal="center", vertical="center")
            
            # TOP 10 (동일한 정렬 기준 적용)
            ws_dashboard['A12'] = "🏆 상위 리셀러 후보 TOP 10"
            ws_dashboard['A12'].font = Font(bold=True, size=14)
            
            # ⭐ 수정된 부분: 일관된 정렬 기준
            top10 = final_results.sort_values(
                ['리셀러점수', '총매출액', '주문횟수'], 
                ascending=[False, False, False]
            ).head(10)
            
            top10_data = [["순위", "고객명", "점수", "등급", "총매출", "주문횟수"]]
            
            for rank, (idx, row) in enumerate(top10.iterrows(), start=1):
                top10_data.append([
                    rank, idx, f"{row['리셀러점수']:.0f}", row['등급'],
                    f"{row['총매출액']:,.0f}", row['주문횟수']
                ])
            
            for r_idx, row in enumerate(top10_data, start=14):
                for c_idx, value in enumerate(row, start=1):
                    cell = ws_dashboard.cell(row=r_idx, column=c_idx, value=value)
                    if r_idx == 14:
                        cell.fill = header_fill
                        cell.font = header_font
            
            # ===== 차트 이미지 삽입 =====
            try:
                # 차트 1: 등급별 분포 (F3 위치)
                img1 = Image(chart_files[0])
                img1.width = 300
                img1.height = 250
                ws_dashboard.add_image(img1, 'F3')
                
                # 차트 2: TOP 10 점수 (F15 위치)  
                img2 = Image(chart_files[1])
                img2.width = 400
                img2.height = 300
                ws_dashboard.add_image(img2, 'F15')
                
                # 차트 3: 매출 vs 주문횟수 (M3 위치)
                img3 = Image(chart_files[2])
                img3.width = 400
                img3.height = 300
                ws_dashboard.add_image(img3, 'M3')
                
                # 차트 4: 점수 구성 비교 (M15 위치)
                img4 = Image(chart_files[3])
                img4.width = 400
                img4.height = 300
                ws_dashboard.add_image(img4, 'M15')
                
                print("✅ 차트 이미지 삽입 완료!")
                
            except Exception as e:
                print(f"⚠️ 차트 삽입 중 오류: {e}")
            
            # ===== 시트2: 전체 고객 분석 =====
            ws_detail = wb.create_sheet(title="📋 전체 고객 분석")
            
            # ⭐ 수정된 부분: 일관된 정렬 기준
            export_data = final_results.sort_values(
                ['리셀러점수', '총매출액', '주문횟수'], 
                ascending=[False, False, False]
            )
            export_columns = [
                '등급', '판정', '리셀러점수',
                '총구매수량', '평균구매수량', '주문횟수',
                '총매출액', '구매제품종류',
                '사업장주소', '활동기간(일)', '평균구매주기'
            ]
            
            # 헤더
            headers = ['고객명'] + export_columns
            for c_idx, header in enumerate(headers, start=1):
                cell = ws_detail.cell(row=1, column=c_idx, value=header)
                cell.fill = header_fill
                cell.font = header_font
            
            # 데이터
            for r_idx, (idx, row) in enumerate(export_data[export_columns].iterrows(), start=2):
                ws_detail.cell(row=r_idx, column=1, value=idx)
                for c_idx, col in enumerate(export_columns, start=2):
                    value = row[col]
                    cell = ws_detail.cell(row=r_idx, column=c_idx, value=value)
                    
                    # 등급별 색상
                    if col == '등급':
                        if value == 'A':
                            cell.fill = PatternFill(start_color="FF6B6B", end_color="FF6B6B", fill_type="solid")
                        elif value == 'B':
                            cell.fill = PatternFill(start_color="FFA500", end_color="FFA500", fill_type="solid")
                        elif value == 'C':
                            cell.fill = PatternFill(start_color="FFD700", end_color="FFD700", fill_type="solid")
            
            # ===== 시트3: VIP 리셀러 =====
            ws_vip = wb.create_sheet(title="⭐ VIP 리셀러")
            
            vip_data = export_data[export_data['등급'].isin(['A', 'B'])]
            vip_headers = ['고객명', '등급', '점수', '총매출액', '주문횟수', '평균구매수량', '주력상품TOP3']
            
            for c_idx, header in enumerate(vip_headers, start=1):
                cell = ws_vip.cell(row=1, column=c_idx, value=header)
                cell.fill = PatternFill(start_color="FFD700", end_color="FFD700", fill_type="solid")
                cell.font = Font(bold=True, size=12)
            
            for r_idx, (idx, row) in enumerate(vip_data.iterrows(), start=2):
                ws_vip.cell(row=r_idx, column=1, value=idx)
                ws_vip.cell(row=r_idx, column=2, value=row['등급'])
                ws_vip.cell(row=r_idx, column=3, value=f"{row['리셀러점수']:.0f}")
                ws_vip.cell(row=r_idx, column=4, value=row['총매출액'])
                ws_vip.cell(row=r_idx, column=5, value=row['주문횟수'])
                ws_vip.cell(row=r_idx, column=6, value=f"{row['평균구매수량']:.1f}")
                ws_vip.cell(row=r_idx, column=7, value=row['주력상품TOP3'][:50])
            
            # 임시 차트 파일 삭제
            for chart_file in chart_files:
                try:
                    os.remove(chart_file)
                except:
                    pass
            
            return wb

        # 3. 분석 실행
        print("🔍 리셀러 패턴 분석 중...")
        customer_analysis = analyze_reseller_patterns(df)
        final_results = calculate_professional_reseller_score(customer_analysis)

        # 4. 통계 출력
        total = len(final_results)
        a_grade = len(final_results[final_results['등급'] == 'A'])
        b_grade = len(final_results[final_results['등급'] == 'B'])
        c_grade = len(final_results[final_results['등급'] == 'C'])
        d_grade = total - a_grade - b_grade - c_grade

        print("\n📊 리셀러 분포:")
        print(f"🔴 A등급 (확실한 리셀러): {a_grade}명 ({a_grade/total*100:.1f}%)")
        print(f"🟠 B등급 (가능성 높음): {b_grade}명 ({b_grade/total*100:.1f}%)")
        print(f"🟡 C등급 (잠재 리셀러): {c_grade}명 ({c_grade/total*100:.1f}%)")
        print(f"🟢 D등급 (일반 고객): {d_grade}명 ({d_grade/total*100:.1f}%)")
        
        # 5. TOP 10 출력 (일관된 정렬 기준)
        print("\n" + "=" * 60)
        print("🔥 상위 리셀러 TOP 10")
        print("=" * 60)
        
        # ⭐ 수정된 부분: 일관된 정렬 기준
        top_resellers = final_results.sort_values(
            ['리셀러점수', '총매출액', '주문횟수'], 
            ascending=[False, False, False]
        ).head(10)
        
        for rank, (idx, row) in enumerate(top_resellers.iterrows(), 1):
            print(f"\n{rank}위. {idx}")
            print(f"   📊 등급: {row['판정']}")
            print(f"   🎯 점수: {row['리셀러점수']:.0f}점")
            print(f"   💰 매출: {row['총매출액']:,.0f}원")
            print(f"   📦 주문: {row['주문횟수']}회, 평균 {row['평균구매수량']:.1f}개")
            print(f"   🛍️ 주력: {row['주력상품TOP3'][:50]}...")

        # 6. 시각화 (한글 폰트 적용)
        print("\n📊 분석 차트 생성 중...")
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('리셀러 분석 대시보드', fontsize=16, fontweight='bold')

        # 1. 등급별 분포 (파이 차트)
        ax1 = axes[0, 0]
        grade_counts = final_results['등급'].value_counts()
        colors = ['#FF6B6B', '#FFA500', '#FFD700', '#90EE90']
        ax1.pie(grade_counts.values, labels=[f'{k}등급\n({v}명)' for k, v in grade_counts.items()], 
                colors=colors, autopct='%1.1f%%', startangle=90)
        ax1.set_title('리셀러 등급 분포')

        # 2. 상위 10명 점수 (막대 그래프) - 일관된 정렬
        ax2 = axes[0, 1]
        top10_chart = final_results.sort_values(
            ['리셀러점수', '총매출액', '주문횟수'], 
            ascending=[False, False, False]
        ).head(10)
        ax2.barh(range(len(top10_chart)), top10_chart['리셀러점수'], color='steelblue')
        ax2.set_yticks(range(len(top10_chart)))
        ax2.set_yticklabels(top10_chart.index)
        ax2.set_xlabel('리셀러 점수')
        ax2.set_title('TOP 10 리셀러 점수')
        ax2.invert_yaxis()

        # 3. 매출 vs 주문횟수 (산점도)
        ax3 = axes[1, 0]
        scatter = ax3.scatter(final_results['총매출액']/10000, final_results['주문횟수'], 
                             c=final_results['리셀러점수'], cmap='RdYlGn', 
                             alpha=0.6, s=50)
        ax3.set_xlabel('총매출액 (만원)')
        ax3.set_ylabel('주문횟수')
        ax3.set_title('매출 vs 주문횟수')
        plt.colorbar(scatter, ax=ax3)

        # 4. 점수 구성 요소 평균
        ax4 = axes[1, 1]
        score_components = ['대량구매', '정기구매', '사업장', '다양성', '구매규모']
        ab_grade_avg = final_results[final_results['등급'].isin(['A', 'B'])][score_components].mean()
        cd_grade_avg = final_results[final_results['등급'].isin(['C', 'D'])][score_components].mean()

        x = range(len(score_components))
        width = 0.35
        ax4.bar([i - width/2 for i in x], ab_grade_avg, width, label='A/B등급', color='#FF6B6B')
        ax4.bar([i + width/2 for i in x], cd_grade_avg, width, label='C/D등급', color='#FFD700')
        ax4.set_xticks(x)
        ax4.set_xticklabels(score_components, rotation=45)
        ax4.set_ylabel('평균 점수')
        ax4.set_title('등급별 점수 구성 비교')
        ax4.legend()

        plt.tight_layout()
        plt.show()

        # 7. 엑셀 저장 (시각화 포함)
        print("\n" + "=" * 60)
        print("💾 프리미엄 엑셀 리포트 생성 중 (시각화 포함)...")
        
        wb = create_premium_excel_report(final_results)
        output_filename = '리셀러_프리미엄_분석_결과.xlsx'
        wb.save(output_filename)
        print(f"✅ 프리미엄 리포트 생성 완료: {output_filename}")
        print("   📊 대시보드: 전체 통계, TOP 10, 4개 시각화 차트")
        print("   📋 전체 분석: 모든 고객 상세 (색상, 조건부 서식)")
        print("   ⭐ VIP 리셀러: A/B등급 고객만 별도 관리")
        
        # 8. 다운로드
        try:
            from google.colab import files
            print("\n📥 다운로드 시작...")
            files.download(output_filename)
            print("\n🎉 완료! 브라우저의 다운로드 폴더를 확인하세요.")
        except:
            print(f"\n💾 파일이 저장되었습니다: {output_filename}")
            print("   왼쪽 사이드바 📁에서 다운로드하세요.")
            
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("\n확인사항:")
    print("1. 파일명: erp_order_form.xlsx")
    print("2. 시트명: 오전, 오후")
    print("3. 필수 컬럼: 주문자명, 주문수량, 총매출액, 주문일자, 상품명, 주소")